In [212]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [170]:
df=pd.read_csv('/Users/chauhanabhishek/Downloads/music_genre.csv')

In [171]:
df.sample(5)

,instance_id,artist_name,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,obtained_date,valence,music_genre
9120,23637.0,Akihiko Matsumoto,Letter,8.0,0.99100,0.396,210187.0,0.00627,0.883000,A,0.0808,-23.327,Minor,0.0633,133.42600000000002,4-Apr,0.340,Anime
23550,62775.0,empty_field,Ain't Killed Me Yet,47.0,0.04370,0.613,202787.0,0.94100,0.000008,F,0.0760,-3.355,Major,0.0506,?,4-Apr,0.626,Country
15227,23229.0,Mindless Self Indulgence,Get It Up,42.0,0.00949,0.654,155760.0,0.89800,0.000000,A,0.3140,-3.966,Minor,0.0682,119.97200000000001,4-Apr,0.473,Alternative
15383,55746.0,empty_field,Barefoot In The Park (feat. ROSALÍA),63.0,0.83200,0.558,211200.0,0.63200,0.105000,A,0.1980,-8.339,Minor,0.2840,161.764,4-Apr,0.519,Alternative
41250,30224.0,Wolfgang Amadeus Mozart,"Violin Concerto No. 1 in B-Flat Major, K. 207 ...",36.0,0.96800,0.433,447999.0,0.10900,0.589000,A#,0.0919,-22.194,Major,0.0558,?,4-Apr,0.397,Classical


In [172]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50005 entries, 0 to 50004
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   instance_id       50000 non-null  float64
 1   artist_name       50000 non-null  object 
 2   track_name        50000 non-null  object 
 3   popularity        50000 non-null  float64
 4   acousticness      50000 non-null  float64
 5   danceability      50000 non-null  float64
 6   duration_ms       50000 non-null  float64
 7   energy            50000 non-null  float64
 8   instrumentalness  50000 non-null  float64
 9   key               50000 non-null  object 
 10  liveness          50000 non-null  float64
 11  loudness          50000 non-null  float64
 12  mode              50000 non-null  object 
 13  speechiness       50000 non-null  float64
 14  tempo             50000 non-null  object 
 15  obtained_date     50000 non-null  object 
 16  valence           50000 non-null  float6

In [173]:
df.describe()

,instance_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,valence
count,50000.000000,50000.000000,50000.000000,50000.000000,5.000000e+04,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,55888.396360,44.220420,0.306383,0.558241,2.212526e+05,0.599755,0.181601,0.193896,-9.133761,0.093586,0.456264
std,20725.256253,15.542008,0.341340,0.178632,1.286720e+05,0.264559,0.325409,0.161637,6.162990,0.101373,0.247119
min,20002.000000,0.000000,0.000000,0.059600,-1.000000e+00,0.000792,0.000000,0.009670,-47.046000,0.022300,0.000000
25%,37973.500000,34.000000,0.020000,0.442000,1.748000e+05,0.433000,0.000000,0.096900,-10.860000,0.036100,0.257000
50%,55913.500000,45.000000,0.144000,0.568000,2.192810e+05,0.643000,0.000158,0.126000,-7.276500,0.048900,0.448000
75%,73863.250000,56.000000,0.552000,0.687000,2.686122e+05,0.815000,0.155000,0.244000,-5.173000,0.098525,0.648000
max,91759.000000,99.000000,0.996000,0.986000,4.830606e+06,0.999000,0.996000,1.000000,3.744000,0.942000,0.992000


In [174]:
df.head()

,instance_id,artist_name,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,obtained_date,valence,music_genre
0,32894.0,Röyksopp,Röyksopp's Night Out,27.0,0.00468,0.652,-1.0,0.941,0.79200,A#,0.115,-5.201,Minor,0.0748,100.889,4-Apr,0.759,Electronic
1,46652.0,Thievery Corporation,The Shining Path,31.0,0.01270,0.622,218293.0,0.890,0.95000,D,0.124,-7.043,Minor,0.0300,115.00200000000001,4-Apr,0.531,Electronic
2,30097.0,Dillon Francis,Hurricane,28.0,0.00306,0.620,215613.0,0.755,0.01180,G#,0.534,-4.617,Major,0.0345,127.994,4-Apr,0.333,Electronic
3,62177.0,Dubloadz,Nitro,34.0,0.02540,0.774,166875.0,0.700,0.00253,C#,0.157,-4.498,Major,0.2390,128.014,4-Apr,0.270,Electronic
4,24907.0,What So Not,Divide & Conquer,32.0,0.00465,0.638,222369.0,0.587,0.90900,F#,0.157,-6.266,Major,0.0413,145.036,4-Apr,0.323,Electronic


In [175]:
df.drop(columns=["instance_id","artist_name","track_name","obtained_date"],inplace=True)

In [176]:
df

,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,music_genre
0,27.0,0.00468,0.652,-1.0,0.941,0.79200,A#,0.115,-5.201,Minor,0.0748,100.889,0.759,Electronic
1,31.0,0.01270,0.622,218293.0,0.890,0.95000,D,0.124,-7.043,Minor,0.0300,115.00200000000001,0.531,Electronic
2,28.0,0.00306,0.620,215613.0,0.755,0.01180,G#,0.534,-4.617,Major,0.0345,127.994,0.333,Electronic
3,34.0,0.02540,0.774,166875.0,0.700,0.00253,C#,0.157,-4.498,Major,0.2390,128.014,0.270,Electronic
4,32.0,0.00465,0.638,222369.0,0.587,0.90900,F#,0.157,-6.266,Major,0.0413,145.036,0.323,Electronic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50000,59.0,0.03340,0.913,-1.0,0.574,0.00000,C#,0.119,-7.022,Major,0.2980,98.02799999999999,0.330,Hip-Hop
50001,72.0,0.15700,0.709,251860.0,0.362,0.00000,B,0.109,-9.814,Major,0.0550,122.04299999999999,0.113,Hip-Hop
50002,51.0,0.00597,0.693,189483.0,0.763,0.00000,D,0.143,-5.443,Major,0.1460,131.079,0.395,Hip-Hop
50003,65.0,0.08310,0.782,262773.0,0.472,0.00000,G,0.106,-5.016,Minor,0.0441,75.88600000000001,0.354,Hip-Hop


In [177]:
df["key"].nunique()

12

In [178]:
df["key"].value_counts()

G     5727
C     5522
C#    5405
D     5265
A     4825
F     4341
B     3789
E     3760
A#    3356
G#    3319
F#    3101
D#    1590
Name: key, dtype: int64

In [83]:
df.corr()

,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,valence
popularity,1.000000,-0.290453,0.356420,-0.083809,0.216345,-0.364960,-0.039468,0.317941,0.224309,0.124913
acousticness,-0.290453,1.000000,-0.347681,0.061862,-0.791250,0.387970,-0.109220,-0.730401,-0.163377,-0.270238
danceability,0.356420,-0.347681,1.000000,-0.155507,0.263934,-0.301277,-0.051465,0.390859,0.279767,0.434557
duration_ms,-0.083809,0.061862,-0.155507,1.000000,-0.066203,0.129738,0.019911,-0.102509,-0.083636,-0.130574
energy,0.216345,-0.791250,0.263934,-0.066203,1.000000,-0.389730,0.186731,0.838399,0.158180,0.396317
instrumentalness,-0.364960,0.387970,-0.301277,0.129738,-0.389730,1.000000,-0.093537,-0.529030,-0.199137,-0.272780
liveness,-0.039468,-0.109220,-0.051465,0.019911,0.186731,-0.093537,1.000000,0.126504,0.114475,0.052110
loudness,0.317941,-0.730401,0.390859,-0.102509,0.838399,-0.529030,0.126504,1.000000,0.165578,0.340664
speechiness,0.224309,-0.163377,0.279767,-0.083636,0.158180,-0.199137,0.114475,0.165578,1.000000,0.059847
valence,0.124913,-0.270238,0.434557,-0.130574,0.396317,-0.272780,0.052110,0.340664,0.059847,1.000000


In [84]:
from pandas_profiling import ProfileReport
prof=ProfileReport(df)
prof.to_file(output_file="output.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/scipy/cluster/hierarchy.py:2844: UserWarning: Attempting to set identical bottom == top == 0.0 results in singular transformations; automatically expanding.
  ax.set_ylim([dvw, 0])


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [179]:
df.drop(columns="instrumentalness",inplace=True)

In [180]:
df.drop_duplicates(keep="first",inplace=True)

In [181]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 50004
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   popularity    49999 non-null  float64
 1   acousticness  49999 non-null  float64
 2   danceability  49999 non-null  float64
 3   duration_ms   49999 non-null  float64
 4   energy        49999 non-null  float64
 5   key           49999 non-null  object 
 6   liveness      49999 non-null  float64
 7   loudness      49999 non-null  float64
 8   mode          49999 non-null  object 
 9   speechiness   49999 non-null  float64
 10  tempo         49999 non-null  object 
 11  valence       49999 non-null  float64
 12  music_genre   49999 non-null  object 
dtypes: float64(9), object(4)
memory usage: 5.3+ MB


In [182]:
df

,popularity,acousticness,danceability,duration_ms,energy,key,liveness,loudness,mode,speechiness,tempo,valence,music_genre
0,27.0,0.00468,0.652,-1.0,0.941,A#,0.115,-5.201,Minor,0.0748,100.889,0.759,Electronic
1,31.0,0.01270,0.622,218293.0,0.890,D,0.124,-7.043,Minor,0.0300,115.00200000000001,0.531,Electronic
2,28.0,0.00306,0.620,215613.0,0.755,G#,0.534,-4.617,Major,0.0345,127.994,0.333,Electronic
3,34.0,0.02540,0.774,166875.0,0.700,C#,0.157,-4.498,Major,0.2390,128.014,0.270,Electronic
4,32.0,0.00465,0.638,222369.0,0.587,F#,0.157,-6.266,Major,0.0413,145.036,0.323,Electronic
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50000,59.0,0.03340,0.913,-1.0,0.574,C#,0.119,-7.022,Major,0.2980,98.02799999999999,0.330,Hip-Hop
50001,72.0,0.15700,0.709,251860.0,0.362,B,0.109,-9.814,Major,0.0550,122.04299999999999,0.113,Hip-Hop
50002,51.0,0.00597,0.693,189483.0,0.763,D,0.143,-5.443,Major,0.1460,131.079,0.395,Hip-Hop
50003,65.0,0.08310,0.782,262773.0,0.472,G,0.106,-5.016,Minor,0.0441,75.88600000000001,0.354,Hip-Hop


In [183]:
df = df.replace("?",np.nan)

In [184]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 50004
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   popularity    49999 non-null  float64
 1   acousticness  49999 non-null  float64
 2   danceability  49999 non-null  float64
 3   duration_ms   49999 non-null  float64
 4   energy        49999 non-null  float64
 5   key           49999 non-null  object 
 6   liveness      49999 non-null  float64
 7   loudness      49999 non-null  float64
 8   mode          49999 non-null  object 
 9   speechiness   49999 non-null  float64
 10  tempo         45019 non-null  object 
 11  valence       49999 non-null  float64
 12  music_genre   49999 non-null  object 
dtypes: float64(9), object(4)
memory usage: 5.3+ MB


In [185]:
df["tempo"]=df["tempo"].astype(float)

In [186]:
df.isnull().sum()

popularity         1
acousticness       1
danceability       1
duration_ms        1
energy             1
key                1
liveness           1
loudness           1
mode               1
speechiness        1
tempo           4981
valence            1
music_genre        1
dtype: int64

In [187]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 50004
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   popularity    49999 non-null  float64
 1   acousticness  49999 non-null  float64
 2   danceability  49999 non-null  float64
 3   duration_ms   49999 non-null  float64
 4   energy        49999 non-null  float64
 5   key           49999 non-null  object 
 6   liveness      49999 non-null  float64
 7   loudness      49999 non-null  float64
 8   mode          49999 non-null  object 
 9   speechiness   49999 non-null  float64
 10  tempo         45019 non-null  float64
 11  valence       49999 non-null  float64
 12  music_genre   49999 non-null  object 
dtypes: float64(10), object(3)
memory usage: 5.3+ MB


In [188]:
ef=df.drop(columns=["key","mode","music_genre"])

In [189]:
ii=IterativeImputer()
transfromed=ii.fit_transform(ef)
ef=pd.DataFrame(transfromed,columns=["popularity","acousticness","danceability","duration_ms","energy","liveness","loudness","speechiness","tempo","valence"])


In [190]:
aa=df.loc[:,["key","mode","music_genre"]]

In [191]:
df=pd.concat([ef,aa],axis=1)

In [192]:
df

,popularity,acousticness,danceability,duration_ms,energy,liveness,loudness,speechiness,tempo,valence,key,mode,music_genre
0,27.0,0.00468,0.652,-1.0,0.941,0.115,-5.201,0.0748,100.889,0.759,A#,Minor,Electronic
1,31.0,0.01270,0.622,218293.0,0.890,0.124,-7.043,0.0300,115.002,0.531,D,Minor,Electronic
2,28.0,0.00306,0.620,215613.0,0.755,0.534,-4.617,0.0345,127.994,0.333,G#,Major,Electronic
3,34.0,0.02540,0.774,166875.0,0.700,0.157,-4.498,0.2390,128.014,0.270,C#,Major,Electronic
4,32.0,0.00465,0.638,222369.0,0.587,0.157,-6.266,0.0413,145.036,0.323,F#,Major,Electronic
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C#,Major,Hip-Hop
50001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,Major,Hip-Hop
50002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,Major,Hip-Hop
50003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G,Minor,Hip-Hop


In [193]:
df.dropna(inplace=True)

In [194]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49994 entries, 0 to 49999
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   popularity    49994 non-null  float64
 1   acousticness  49994 non-null  float64
 2   danceability  49994 non-null  float64
 3   duration_ms   49994 non-null  float64
 4   energy        49994 non-null  float64
 5   liveness      49994 non-null  float64
 6   loudness      49994 non-null  float64
 7   speechiness   49994 non-null  float64
 8   tempo         49994 non-null  float64
 9   valence       49994 non-null  float64
 10  key           49994 non-null  object 
 11  mode          49994 non-null  object 
 12  music_genre   49994 non-null  object 
dtypes: float64(10), object(3)
memory usage: 5.3+ MB


In [195]:
x=df.iloc[:,:12]
y=df.iloc[:,12]

In [196]:
le=LabelEncoder()

In [197]:
y=le.fit_transform(y)

In [198]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49994 entries, 0 to 49999
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   popularity    49994 non-null  float64
 1   acousticness  49994 non-null  float64
 2   danceability  49994 non-null  float64
 3   duration_ms   49994 non-null  float64
 4   energy        49994 non-null  float64
 5   liveness      49994 non-null  float64
 6   loudness      49994 non-null  float64
 7   speechiness   49994 non-null  float64
 8   tempo         49994 non-null  float64
 9   valence       49994 non-null  float64
 10  key           49994 non-null  object 
 11  mode          49994 non-null  object 
dtypes: float64(10), object(2)
memory usage: 5.0+ MB


In [199]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.15)

In [200]:
transformer = ColumnTransformer(transformers=[
    ("tnf1",OneHotEncoder(sparse=False,drop="first"),[10,11]),
],remainder='passthrough')

In [214]:
model=RandomForestClassifier(
    n_estimators=100,
    random_state=3,
    max_samples=0.5,
    max_features=0.75,
    max_depth=15
)

In [215]:
pipe=Pipeline([
    ("step1",transformer),
    ("step2",model)
])

In [216]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('step1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('tnf1',
                                                  OneHotEncoder(drop='first',
                                                                sparse=False),
                                                  [10, 11])])),
                ('step2',
                 RandomForestClassifier(max_depth=15, max_features=0.75,
                                        max_samples=0.5, random_state=3))])

In [217]:
y_pred=pipe.predict(x_test)

In [219]:
print("Accuracy Score=",accuracy_score(y_test,y_pred))

Accuracy Score= 0.5325333333333333
